# Lesson 2.1:
# PowerGrid Models API

This tutorial introduces the PowerGrid Models API and how it can be used to query model data.

__Learning Objectives:__

At the end of the tutorial, the user should be able to use the PowerGrid Models API to

* 
* 
* 

## Getting Started

Before running any of the sample routines in this tutorial, it is first necessary to start the GridAPPS-D Platform and establish a connection to this notebook so that we can start passing calls to the API.

_Open the Ubuntu terminal and start the GridAPPS-D Platform if it is not running already:_

`cd gridappsd-docker`

~/gridappsd-docker$ `./run.sh -t develop`

_Once containers are running,_

gridappsd@[container]:/gridappsd$ `./run-gridappsd.sh`

In [19]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
gapps = GridAPPSD("('localhost', 61613)", username='system', password='manager')

INFO:goss:Starting thread queue


# 1. Blazegraph Database

GridAPPS-D uses the Blazegraph database to store power system models. 

___INSERT MORE ON BLAZEGRAPH___

___INSERT MORE ON BLAZEGRAPH___

___INSERT MORE ON BLAZEGRAPH___

___INSERT MORE ON BLAZEGRAPH___

# 2. Using GridAPPSD-Python Functions

The GridAPPSD-Python library contains several pre-built functions to perform simple queries of the Blazegraph database to obtain commonly used power system model information.

## 2.1. Querying for All _modelNames_ mRIDs

When passing commands through the API, it is often necessary to specify the MRID of the particular power system network model.

The PowerGrid Models API containes a function to obtain a list of all the model MRIDs stored in the Blazegrpah database. 

This method will return identical results to the JSON message explained below in section 3.

In [ ]:
gridappsd_conn.query_model_names()

---
# 3. Simple JSON Queries

For more complex API queries, there are some common simple JSON requests that we can pass 

## 3.1. Specifying the Topic

All queries passed to the PowerGrid Models API need to use the correct topic. There are two ways to specify the topic:

__1) Specifying the topic as a string:__

In [ ]:
topic = "goss.gridappsd.process.request.data.powergridmodel"

__2) Using the `topics` library to specify the topic:__

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

## 3.xx. Query for All Model Info

This query returns a list of names and MRIDs for all models, substations, subregions, and regions for all available feeders.



In [ ]:
message = '{"requestType": "QUERY_MODEL_INFO","resultFormat": "JSON"}'
gridappsd_conn.get_response(topic, message)

---
## Querying for Model Name mRIDs

When passing commands through the API, it is often necessary to specify the MRID of the particular power system network model.

The PowerGrid Models API containes a function to obtain a list of all the model MRIDs stored in the Blazegrpah database. 

This method will return identical results to the GridAPPSD-Python library function of section  .

In [ ]:
message = '{"requestType": "QUERY_MODEL_NAMES", "stationName": "IEEE13", "resultFormat": "JSON"}'
gridappsd_conn.get_response(topic, message)

## Query for Measurement mRIDs



In [ ]:
message = '''{
        "requestType": "QUERY_OBJECT",
        "resultFormat": "JSON",
        "objectId": "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"
}'''

gridappsd_conn.get_response(topic, message)

---
# 5. Generic SPARQL Queries

In [34]:
query = """
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?type ?name ?id WHERE {
 ?s c:IdentifiedObject.name ?name.
 ?s c:IdentifiedObject.mRID ?id.
 ?s r:type ?rawtype.
  bind(strafter(str(?rawtype),"#") as ?type)
}
ORDER by ?type ?name     
    """ % model_mrid

TypeError: not all arguments converted during string formatting

In [37]:
gapps.query_data(query)

{'data': {'head': {'vars': ['feeder', 'busname', 'cnid', 'nomv']},
  'results': {'bindings': [{'feeder': {'type': 'literal',
      'value': 'acep_psil'},
     'busname': {'type': 'literal', 'value': 'bank1'},
     'cnid': {'type': 'literal',
      'value': '_645CD885-C460-46B7-B534-32D3D613E8A6'},
     'nomv': {'type': 'literal', 'value': '208'}},
    {'feeder': {'type': 'literal', 'value': 'acep_psil'},
     'busname': {'type': 'literal', 'value': 'batt'},
     'cnid': {'type': 'literal',
      'value': '_DBA21209-E372-4224-8058-EB89F650D47D'},
     'nomv': {'type': 'literal', 'value': '480'}},
    {'feeder': {'type': 'literal', 'value': 'acep_psil'},
     'busname': {'type': 'literal', 'value': 'gen1'},
     'cnid': {'type': 'literal',
      'value': '_A38739BE-985A-400E-A44E-BBCF5F56478C'},
     'nomv': {'type': 'literal', 'value': '480'}},
    {'feeder': {'type': 'literal', 'value': 'acep_psil'},
     'busname': {'type': 'literal', 'value': 'load1'},
     'cnid': {'type': 'literal'

In [ ]:
query = """
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT DISTINCT ?name ?id WHERE {
 ?crs r:type c:CoordinateSystem.
 ?crs c:IdentifiedObject.mRID ?id.
 ?crs c:IdentifiedObject.name ?name.
}
ORDER by ?name


In [52]:
query = """
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT DISTINCT ?feeder ?busname ?cnid ?nomv WHERE {
    VALUES ?fdrid {"%s"}  # inserts model_mrid
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    ?bus c:ConnectivityNode.ConnectivityNodeContainer ?fdr.
    ?bus r:type c:ConnectivityNode.
    ?bus c:IdentifiedObject.name ?busname.
    ?bus c:IdentifiedObject.mRID ?cnid.
    ?fdr c:IdentifiedObject.name ?feeder.
    ?trm c:Terminal.ConnectivityNode ?bus.
    ?trm c:Terminal.ConductingEquipment ?ce.
    ?ce  c:ConductingEquipment.BaseVoltage ?bv.
    ?bv  c:BaseVoltage.nominalVoltage ?nomv.
    }
    ORDER by ?feeder ?busname ?nomv
    """ % model_mrid

In [43]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node

In [38]:
query = """
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT ?cimtype ?name ?bus1 ?bus2 ?id WHERE {
    SELECT ?cimtype ?name ?bus1 ?bus2 ?phs ?id WHERE {
    VALUES ?fdrid {"%s"}  # 9500 node
    VALUES ?cimraw {c:LoadBreakSwitch c:Recloser c:Breaker}
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    ?s r:type ?cimraw.
    bind(strafter(str(?cimraw),"#") as ?cimtype)
    ?s c:Equipment.EquipmentContainer ?fdr.
    ?s c:IdentifiedObject.name ?name.
    ?s c:IdentifiedObject.mRID ?id.
    ?t1 c:Terminal.ConductingEquipment ?s.
    ?t1 c:ACDCTerminal.sequenceNumber "1".
    ?t1 c:Terminal.ConnectivityNode ?cn1. 
    ?cn1 c:IdentifiedObject.name ?bus1.
    ?t2 c:Terminal.ConductingEquipment ?s.
    ?t2 c:ACDCTerminal.sequenceNumber "2".
    ?t2 c:Terminal.ConnectivityNode ?cn2. 
    ?cn2 c:IdentifiedObject.name ?bus2
        OPTIONAL {?swp c:SwitchPhase.Switch ?s.
        ?swp c:SwitchPhase.phaseSide1 ?phsraw.
        bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }
    } ORDER BY ?name ?phs
    }
    GROUP BY ?cimtype ?name ?bus1 ?bus2 ?id
    ORDER BY ?cimtype ?name
        """ % model_mrid

In [39]:
gapps.query_data(query)

{'data': {'head': {'vars': ['cimtype', 'name', 'bus1', 'bus2', 'id']},
  'results': {'bindings': [{'cimtype': {'type': 'literal', 'value': 'Breaker'},
     'name': {'type': 'literal', 'value': 'brkr1'},
     'bus1': {'type': 'literal', 'value': '650'},
     'bus2': {'type': 'literal', 'value': 'brkr'},
     'id': {'type': 'literal',
      'value': '_52DE9189-20DC-4C73-BDEE-E960FE1F9493'}},
    {'cimtype': {'type': 'literal', 'value': 'LoadBreakSwitch'},
     'name': {'type': 'literal', 'value': '671692'},
     'bus1': {'type': 'literal', 'value': '671'},
     'bus2': {'type': 'literal', 'value': '692'},
     'id': {'type': 'literal',
      'value': '_517413CB-6977-46FA-8911-C82332E42884'}},
    {'cimtype': {'type': 'literal', 'value': 'LoadBreakSwitch'},
     'name': {'type': 'literal', 'value': 'sect1'},
     'bus1': {'type': 'literal', 'value': '684'},
     'bus2': {'type': 'literal', 'value': 'tap'},
     'id': {'type': 'literal',
      'value': '_2858B6C2-0886-4269-884C-06FA8B88731